In [350]:
class Item:
    def __init__(self, item: str, utility: int):
        self.item = item
        self.utility = utility
        self._twu = 0

    @property
    def twu(self) -> int:
        return self._twu

    @twu.setter
    def twu(self, value: int) -> None:
        if value < 0:
            raise ValueError("TWU value cannot be negative!")
        self._twu = value

    def __repr__(self):
        # return f"({self.item},{self.twu})"
        return f"{self.item}"

In [351]:
class Transaction:
    def __init__(self, id: int, items_quantities: dict):
        if any(q <= 0 for q in items_quantities.values()):
            raise ValueError(f"Quantities in trans{id} must be positive integers.")
        self.id = id
        self.items_quantities = items_quantities

    def __repr__(self):
        return (f"(tid = {self.id}, frequencies = {self.items_quantities})")

## Table 1  
**Database**

| Tid  | Item        | Quantity  |
|------|-------------|-----------|
| T₁   | a b c d     | 5 2 1 2   |
| T₂   | a c d g     | 1 1 1 3   |
| T₃   | a c f       | 1 1 1     |
| T₄   | a f g       | 1 4 2     |
| T₅   | a g         | 1 2       |
| T₆   | b c d e     | 3 2 3 1   |
| T₇   | c e         | 6 4       |
| T₈   | e f         | 1 3       |

---
## Table 2  
**Unit Utility**

| Item   | a | b | c  | d  | e  | f  | g  |
|--------|---|---|----|----|----|----|----|
| Utility| 3 | 6 | -3 | 12 | -5 | -2 | -1 |


In [352]:
a = Item("A", 3)
b = Item("B", 6)
c = Item("C", -3)
d = Item("D", 12)
e = Item("E", -5)
f = Item("F", -2)
g = Item("G", -1)

trans1 = Transaction(1, dict(zip([a, b, c, d], [5, 2, 1, 2])))
trans2 = Transaction(2, dict(zip([a, c, d, g], [1, 1, 1, 3])))
trans3 = Transaction(3, dict(zip([a, c, f], [1, 1, 1])))
trans4 = Transaction(4, dict(zip([a, f, g], [1, 4, 2])))
trans5 = Transaction(5, dict(zip([a, g], [1, 2])))
trans6 = Transaction(6, dict(zip([b, c, d, e], [3, 2, 3, 1])))
trans7 = Transaction(7, dict(zip([c, e], [6, 4])))
trans8 = Transaction(8, dict(zip([e, f], [1, 3])))

items = [a, b, c, d, e, f, g]
database = [trans1, trans2, trans3, trans4, trans5, trans6, trans7, trans8]


Definition 1

In [353]:
def find_all_transactions_contained_item_set(item_set: set[Item], database: list[Transaction]):
    transactionsList: list[Transaction] = []
    for trans in database:
        if item_set.issubset(trans.items_quantities.keys()):
            transactionsList.append(trans)
    return transactionsList

In [354]:
def find_max_min_avg_periodic_of_item_set(item_set: set[Item], database: list[Transaction]):
    contained_transactions = find_all_transactions_contained_item_set(item_set, database)
    trans_ids: list[int] = [trans.id for trans in contained_transactions]
    m = len(contained_transactions)
    ps: list[int] = []
    for i in range(0, m + 1):
        if(i == 0):
            ps.append(trans_ids[0]);
        elif(i == m):
            ps.append(len(database) - trans_ids[m - 1])
        else:
            ps.append(trans_ids[i] - trans_ids[i - 1])
    max_per, min_per = ps[0], ps[0]
    avg_per = len(database) / len(ps)
    for i in ps:
        if(i > max_per):
            max_per = i
        if(i < min_per):
            min_per = i
    return max_per, min_per, avg_per

<center>
Definition 4: The positive utility of X in Transaction T as:
<br></br>
PU(X,T) = Σ<sub>i<sub>k</sub>∈X,P(i<sub>k</sub>)>0</sub>(P(i<sub>k</sub>) * q<sub>k</sub>)
</center>

In [355]:
def calculate_positive_utility_of_item_set_in_trans(items: set[Item], trans: Transaction):
    pu = 0
    if(items.issubset(trans.items_quantities.keys())):
        for item in items:
            utility = item.utility 
            if(utility > 0):
                quantity = trans.items_quantities.get(item);
                if(quantity != None):
                    pu += item.utility * quantity
    return pu;

<center>
Definition 4: The positive utility of X in Database D are:
<br></br>
PU(X) = Σ<sub>(X⊆T<sub>j</sub>,T<sub>j</sub>∈D)</sub>PU(X,T<sub>j</sub>)
</center>

In [356]:
def calculate_positive_utility_of_item_set_in_database(items: set[Item], database: list[Transaction]):
    pu = 0
    for trans in database:
        pu += calculate_positive_utility_of_item_set_in_trans(items, trans)
    return pu

<center> Definition 4: The negative utility of X in Transaction T as:
<br></br>
NU(X,T) = Σ<sub>i<sub>k</sub>∈X,P(i<sub>k</sub>)<0</sub>(P(i<sub>k</sub>) * q<sub>k</sub>)
</center>



In [357]:
def calculate_negative_utility_of_item_set_in_trans(items: set[Item], trans: Transaction):
    nu = 0
    if(items.issubset(trans.items_quantities.keys())):
        for item in items:
            utility = item.utility 
            if(utility < 0):
                quantity = trans.items_quantities.get(item);
                if(quantity != None):
                    nu += item.utility * quantity
    return nu;

<center>
Definition 4: The negative utility of X in Database D are:
<br></br>
NU(X) = Σ<sub>(X⊆T<sub>j</sub>,T<sub>j</sub>∈D)</sub>NU(*X,T<sub>j</sub>)
</center>

In [358]:
def calculate_negative_utility_of_item_set_in_database(items: set[Item], database: list[Transaction]):
    nu = 0
    for trans in database:
        nu += calculate_negative_utility_of_item_set_in_trans(items, trans)
    return nu

In [359]:
def calculate_positive_utility_of_transaction(trans: Transaction):
    pu = 0;
    items : set[Item] = trans.items_quantities.keys()
    for item in items:
        if(item.utility > 0):
            pu += trans.items_quantities.get(item) * item.utility
    return pu;

<center>
Definition 6 (Transaction Weighted Utility TWU in N-database):
<br></br>
TWU(X) = Σ<sub>X⊆T<sub>j</sub>,T<sub>j</sub>∈D</sub>PU(T<sub>j</sub>)
</center>

In [360]:
def calculate_transaction_weight_utility(items: set[Item], database: list[Transaction]):
    twu = 0
    for trans in database:
        trans_items = trans.items_quantities.keys()
        if(items.issubset(trans_items)):
            twu += calculate_positive_utility_of_transaction(trans)
    return twu

In [361]:
#a > b
def check_order_of_two_items(a: Item, b: Item, database: list[Transaction]):
    if(a == b):
        return False
    if(a.utility > 0 and b.utility < 0):
        return True
    if(a.utility * b.utility > 0):
        return calculate_transaction_weight_utility({a}, database) > calculate_transaction_weight_utility({b}, database)
    return False

In [362]:
def check_order_item_and_set(ik: Item, X: set[Item], database: list[Transaction]) -> bool:
    for i in X:
        if(check_order_of_two_items(ik, i, database) == False):
            return False
    return True

<center>
Definition 8: The remaining utility of 𝑋 in T<sub>j</sub>
<br></br>
RU(X) = Σ<sub>X⊆T<sub>j</sub>,T<sub>j</sub>∈D</sub>RU(X, T<sub>j</sub>)</center>

In [363]:
def calculate_remaining_utility_of_item_set_in_transaction(items: set[Item], trans: Transaction, database: list[Transaction]):
    ru = 0
    trans_items: set[Item] = trans.items_quantities.keys()
    if items.issubset(trans_items):
        for item in trans_items:
            # print("check item: " + item.item)
            if(item.utility > 0):
                if(check_order_item_and_set(item, items, database)):
                    # print("check_order_item_and_set TRUE" )
                    # print(
                    #     "twu of "
                    #     + item.item + " = "
                    #     + str(calculate_transaction_weight_utility({item}, database))
                    # )
                    ru += item.utility * trans.items_quantities.get(item)
    return ru

In [364]:
def calculate_remaining_utility_of_item_set_in_database(items: set[Item], database: list[Transaction]):
    ru = 0
    for trans in database:
        ru += calculate_remaining_utility_of_item_set_in_transaction(items, trans, database)
        # print("transID: " + str(trans.id) + ", ru: " + str(calculate_remaining_utility_of_item_set_in_transaction(items, trans, database)))
    return ru

In [365]:
def calculate_utility_of_item_set_in_database(
    items: set[Item], database: list[Transaction]
):
    u = 0
    for trans in database:
        if(items.issubset(trans.items_quantities.keys())):
            for item in items:
                u += item.utility * trans.items_quantities.get(item)
    return u

In [366]:
from itertools import chain

def gather_item_set_from_database(database: list[Transaction]) -> set[Item]:
    return set(chain.from_iterable(trans.items_quantities.keys() for trans in database))

In [367]:
def insert_in_order(sorted_items: list[tuple[Item, int]], new_item: tuple[Item, int]):
    item, twu = new_item
    index = 0
    while index < len(sorted_items) and sorted_items[index][1] < twu:
        index += 1
    sorted_items.insert(index, new_item)

In [368]:
def reorder_database(
    transactions: list[Transaction], sorted_items: list[tuple[Item, int]]
):
    item_order = {item: index for index, (item, _) in enumerate(sorted_items)}
    def sort_key(item):
        return item_order.get(
            item, float("inf")
        ) 
    for trans in transactions:
        sorted_items_in_trans = dict(
            sorted(trans.items_quantities.items(), key=lambda x: sort_key(x[0]))
        )
        trans.items_quantities = sorted_items_in_trans 
    return transactions

In [369]:
def reorder_item_list(
    all_distinct_items: set[Item], sorted_items: list[tuple[Item, int]]
) -> list[Item]:
    item_position = {item: idx for idx, (item, _) in enumerate(sorted_items)}
    return sorted(all_distinct_items, key=lambda x: item_position[x])

In [370]:
def create_eucs_dict(arr: list[Item], database: list[Transaction]) -> dict[frozenset[Item], int]:
    n = len(arr)
    eucs_dict = {}
    for i in range(n):
        for j in range(n):
            if i < j:
                item_pair = frozenset({arr[i], arr[j]})
                twu_value = calculate_transaction_weight_utility(item_pair, database)
                eucs_dict[item_pair] = twu_value
    return eucs_dict

In [371]:
def searching_procedure(
    p,
    pList,
    lists: list[list[Item]],
    minUtil,
    minPer,
    maxPer,
    minAvg,
    maxAvg,
    database: list[Transaction],
    eucs_dict : dict[frozenset[Item], int],
    output: list[set[Item]],
):
    print(lists)
    for i in range(0, len(lists) - 1):
        X: list[Item] = lists[i]
        utility = calculate_utility_of_item_set_in_database(set(X), database)
        remaining_utility = calculate_remaining_utility_of_item_set_in_database(
            set(X), database
        )
        min_per, max_per, avg_per = find_max_min_avg_periodic_of_item_set(
            set(X), database
        )
        if (
            utility >= minUtil
            and min_per >= minPer
            and max_per <= maxPer
            and avg_per >= minAvg
            and avg_per <= maxAvg
        ):
            output.append(X)
        if remaining_utility + utility >= minUtil:
            new_lists = []
            new_period = []
            for j in range(i + 1, len(lists)):
                Y = lists[j]
                key = frozenset({X[-1], Y[-1]})
                twu_value = eucs_dict.get(key)
                if(twu_value > minUtil):
                    print(X[-1].item + "," + Y[-1].item + " passes EUCS condition")

In [372]:
def preparation_procedure(
    database: list[Transaction], 
    all_distinct_items: list[Item],
    minUtility, 
    minPer, 
    maxPer, 
    minAvg, 
    maxAvg
):
    removed_list: list[Item] = list()
    for item in all_distinct_items:
        item_max_per, item_min_per, item_avg_per = (
            find_max_min_avg_periodic_of_item_set({item}, database)
        )
        twu = calculate_transaction_weight_utility({item}, database)
        if (
            twu < minUtility
            or item_min_per < minPer
            or item_max_per > maxPer
            or item_avg_per < minAvg
            or item_avg_per > maxAvg
        ):
            removed_list.append(item)
        else:
            item.twu = twu
    all_distinct_items = [
        item for item in all_distinct_items if item not in removed_list
    ]
    all_distinct_items.sort(key=lambda x: x.twu)
    print(all_distinct_items)
    # print(item.item + ", minPer: " + str(item_min_per) + ", maxPer: " +  str(item_max_per) + ", avgPer: " + str(item_avg_per))
    for trans in database:
        for item in removed_list:
            trans.items_quantities.pop(item, None)
        if len(trans.items_quantities.keys()) == 0:
            database.remove(trans)

    # reorder_database(database, sorted_items)
    lists: list[list[Item]] = [[item] for item in all_distinct_items] #list[set[Item]]

    eucs_dict : dict[frozenset[Item], int] = create_eucs_dict(all_distinct_items, database)
    # print(sorted_items_list)
    # for trans in database:
    #     print(trans)

    # for pair, value in create_eucs_list(sorted_list_to_create_eucs, database).items():
    #     print(f"{pair}: {value}")
    searching_procedure([], None, lists, minUtility, minPer, maxPer, minAvg, maxAvg, database, eucs_dict, [])
    return 0

preparation_procedure(database, items, 30, 1, 5, 1, 3)

[A, B, D, C]
[[A], [B], [D], [C]]
A,B passes EUCS condition
A,D passes EUCS condition
A,C passes EUCS condition
B,D passes EUCS condition
B,C passes EUCS condition
D,C passes EUCS condition


0